In [ ]:
import numpy as np
from collections import deque
from math import *
from itertools import permutations
from random import randint

import copy
import graphviz as gv
import networkx as nx
import pygraphviz as pgv
from networkx.drawing.nx_agraph import graphviz_layout
import tempfile
from PIL import Image
import time
import itertools
import numpy as np
from pathlib import Path
import imageio as io

Eps = '_'
MaxLength = 0

greedy_path_color = '#FF0000'
cycle_color = '#0000FF'
OPT_color = '#FF0000'
greedy_build_color = '#0000FF'
normal_edge_color = '#444444'
falling_color = '#00FF00'

In [ ]:
def overlap(s1, s2):
    for i in reversed(range(1, min(len(s1), len(s2)) + 1)):
        if s1[-i:] == s2[:i]:
            return i
    return 0

def prefix(s1, s2):
    return len(s1) - overlap(s1, s2)

def suffix(s1, s2):
    return len(s2) - overlap(s1, s2)

def leng(st):
    return 0 if st == Eps else len(st)

def make_all_substrings_list(strings):
    all_substrings_set = set()
    for i_str in strings:
        for l in range(len(i_str)+1):
            for r in range(l, len(i_str)+1):
                all_substrings_set.add(i_str[l:r])
    all_substrings = sorted(list(all_substrings_set), key=lambda x: (len(x), x))
    all_substrings[0] = Eps
    return all_substrings

def make_pref_suff_list(strings):
    strings_set = set()
    for i_str in strings:
        for i in range(len(i_str)):
            strings_set.add(i_str[:i])
            strings_set.add(i_str[i:])
    strings_output = sorted(list(strings_set), key=lambda x: (len(x), x))
    strings_output[0] = Eps
    return strings_output

In [ ]:
def edges_out(E_all, v):
    return [e for e in E_all if e[0] == v]
def edges_in(E_all, v):
    return [e for e in E_all if e[1] == v]
def edges_down_out(E_all, v):
    return [e for e in E_all if e[0] == v and leng(e[1]) < leng(v)]
def edges_down_in(E_all, v):
    return [e for e in E_all if e[1] == v and leng(e[0]) < leng(v)]
def ballance(E, v):
    return sum([0 if e[0]==e[1] else 1 if e[0]==v else -1 for e in E if v in e ])
output = '-'#*(sum(len(s) for s in input_strings)+1)
answer = ''
Hint = 'Hint'
OPT_string = 'abbaacdbbabc'
exist = []
cycles = []
def weak_component(V, E, v, marked = False):
    ans = [v]
    que = deque()
    que.append(v)
    E1 = [copy.copy(e) for e in E]
    if marked:
        E1 = [copy.copy(E[i]) for i in range(len(E)) if exist[i]] 
    while len(que) > 0:
        u = que.pop()
        for e in edges_out(E1, u):
            if e[1] in V and not e[1] in ans:
                ans.append(e[1])
                que.appendleft(e[1])
        for e in edges_in(E1, u):
            if e[0] in V and not e[0] in ans:
                ans.append(e[0])
                que.appendleft(e[0])
    return ans

def is_eulerian(V, E):
    if len(V) == len(weak_component(V, E, V[0])):
        imbal_in = []
        imbal_out = []
        for v in V:
            dif = ballance(E, v)
            if dif > 0:
                imbal_in.append(v)
            elif dif < 0:
                imbal_out.append(v)
            if not (len(imbal_in) < 2 and len(imbal_out) < 2):
                return False
        if len(imbal_in) != len(imbal_out):
            return False
        elif len(imbal_in) == 1:
            return ballance(E, imbal_in[0])*ballance(E, imbal_out[0]) == -1
        else:
            return True
    else:
        return False
    
    
HG=pgv.AGraph(directed=True, strict=False)
def make_hierarchical_graph(strings):
    global HG
    HG=pgv.AGraph(directed=True, strict=False)
    
    substrings = make_all_substrings_list(strings)
    
    HG.add_nodes_from(substrings)
    HG.add_node(output)
    #HG.add_node(OPT_string)
    #HG.add_node(Hint)
    for length in range(MaxLength+1)[::-1]:
        subgraph = pgv.AGraph()
        if length == 0:
            subgraph.graph_attr['rank']='source' 
        elif length == MaxLength:
            subgraph.graph_attr['rank']='sink' 
        else:
            subgraph.graph_attr['rank']='same' 
        
        #if length == MaxLength:
        #    subgraph.add_node(OPT_string)
            
        for string in substrings:
            if len(string) == length and string != Eps:
                subgraph.add_node(string)
        #if length == MaxLength:
        if length == 0:
            subgraph.add_node(output)
            #subgraph.add_node(Hint)
            
            subgraph.add_node(Eps)
            
            
        tmp = HG.add_subgraph(subgraph, rank='same')
    for v1 in HG.nodes():
        for v2 in HG.nodes():
            if (v1 != v2):
                if(v1 == Eps and len(v2) == 1) or (v2 == Eps and len(v1) == 1) or (v1[1:] == v2) or (v1 == v2[:-1]):
                    HG.add_edge(v1, v2, color = 'black')
    
    for st in substrings:
        if st in strings:
            HG.get_node(st).attr['shape']='square'
            HG.get_node(st).attr['color']='black'
        else:
            HG.get_node(st).attr['shape']='egg'
            HG.get_node(st).attr['color']='black'
            
    return(HG)

filename = 'img/graph'
images = []
do = True
filenumber = 0
exist = []
V = []
mrk = []
E = []
path = []
OPT = []
fall_edges = []

Hint_colors = [greedy_build_color,
               greedy_path_color,
               OPT_color,
               cycle_color,
               falling_color,
               falling_color]
Hints = ['Build Greedy\'s edges',
         'Find eulerian cycle - Greedy path',
         'Build path of OPT SCS',
         'Cycle cover',
         'Merge cycle cover and OPT path',
         'Falling of edges']
Hint_stage = 0
color_only_fall = False
picture = True

def frame2(typ = 'none'):
    global HG, filenumber, path, V, picture
    E1 = [list(e) for e in E]
    for i in range(len(path)):
        path[i] = Eps if path[i]=='' else path[i]
    for i in range(len(E1)):
        E1[i][0] = Eps if E1[i][0]=='' else E1[i][0]
        E1[i][1] = Eps if E1[i][1]=='' else E1[i][1]
    path = [[path[iv], path[iv + 1]] for iv in range(len(path) - 1)]
    
    for u in HG.nodes():
        HG.get_node(u).attr['color'] = 'black'
        HG.get_node(u).attr['style'] = 'solid'
    HG.get_node(output).attr['color'] = 'white'
    HG.get_node(output).attr['label'] = typ if typ != 'none' else 'HG'
    
    for e in HG.edges():
        HG.get_edge(e[0], e[1]).attr['color'] = normal_edge_color    
    if typ == 'OPT':
        for e in OPT:
            HG.get_edge(e[0], e[1]).attr['color'] += ":"+OPT_color
    if typ == 'GREEDY':
        for e in E1:
            HG.get_edge(e[0], e[1]).attr['color'] += ":"+greedy_build_color
    if typ == 'FALL':
        for e in fall_edges:
            HG.get_edge(e[0], e[1]).attr['color'] += ":"+falling_color
    
    HG.draw('img/' + (typ if typ != 'none' else 'HG')+'.png')
        
def frame(v = '#$%', color = 'goldenrod2'):
    global HG, filenumber, path, V, picture
    if picture:
        E1 = [list(e) for e in E]
        for i in range(len(path)):
            path[i] = Eps if path[i]=='' else path[i]
        for i in range(len(E1)):
            E1[i][0] = Eps if E1[i][0]=='' else E1[i][0]
            E1[i][1] = Eps if E1[i][1]=='' else E1[i][1]
        path = [[path[iv], path[iv + 1]] for iv in range(len(path) - 1)]
        for u in HG.nodes():
            HG.get_node(u).attr['color'] = 'black'
            HG.get_node(u).attr['style'] = 'solid'
        HG.get_node(output).attr['color'] = 'white'
        HG.get_node(output).attr['label'] = answer if answer != '' else 'GREEDY - 3'
        #HG.get_node(OPT_string).attr['color'] = 'white'
        #HG.get_node(OPT_string).attr['fontcolor'] = OPT_color
        #HG.get_node(Hint).attr['fontcolor'] = Hint_colors[Hint_stage]
        #HG.get_node(Hint).attr['color'] = 'white'
        #HG.get_node(Hint).attr['label'] = Hints[Hint_stage]
        #HG.get_node(Hint).attr['pos'] = str(float(HG.get_node(Eps).attr['pos'].split(',')[0])/2)+','+HG.get_node(Eps).attr['pos'].split(',')[1]
        if v != '#$%':
            HG.get_node(v).attr['color'] = color
            HG.get_node(v).attr['style'] = 'filled'

        for e in HG.edges():
            HG.get_edge(e[0], e[1]).attr['color'] = normal_edge_color
        for e in path:
            if HG.get_edge(e[0], e[1]).attr['color'] != greedy_path_color:
                HG.get_edge(e[0], e[1]).attr['color'] = greedy_path_color
            else:
                HG.get_edge(e[0], e[1]).attr['color'] += ":"+greedy_path_color
            E1.remove(e)
        if not color_only_fall:
            for e in cycles:
                HG.get_edge(e[0], e[1]).attr['color'] += ":"+cycle_color
            for e in OPT:
                HG.get_edge(e[0], e[1]).attr['color'] += ":"+OPT_color
            for e in E1:
                HG.get_edge(e[0], e[1]).attr['color'] += ":"+greedy_build_color
        for e in fall_edges:
            HG.get_edge(e[0], e[1]).attr['color'] += ":"+falling_color


        HG.draw(filename+str(filenumber)+'.png')
        images.append(Image.open(filename+str(filenumber)+'.png'))
    filenumber+=1

def pause(steps = 3, v = '#$%', color = 'goldenrod2'):
    return 0
    for i in range(steps):
        frame(v, color)
    
def add_edge(e, extra = False):
    if not e in E or extra:
        E.append(e)
    i0 = V.index(e[0])
    i1 = V.index(e[1])
    if mrk[i0] == 0:
        mrk[i0] = mrk[i1]
    elif mrk[i1] == 0:
        mrk[i1] = mrk[i0]
    else:
        col = int(mrk[i0])
        for i in range(len(mrk)):
            if mrk[i] == col:
                mrk[i] = mrk[i1]
def path_summ(a, path):
    s = 0
    for i in range(len(path)-1):
        s += a[path[i]][path[i+1]]
    return s

def max_ATSP(a):
    n = len(a)
    ans_val = 0
    ans_path = []
    for per in permutations(range(n)):
        cur_val = 0
        for v in range(n - 1):
            cur_val += a[per[v]][per[v+1]]
        if(ans_val < cur_val):
            ans_val = cur_val
            ans_path = per
    return ans_path

def make_overlap_graph(strings):
    n = len(strings)
    a = np.zeros([n, n])
    for i in range(n):
        for j in range(n):
            str1 = strings[i]
            str2 = strings[j]
            overlap = 0
            for l in range(min(len(str1), len(str2)) + 1):
                if(str1.endswith(str2[:l])):
                    overlap = l
            a[i][j] = overlap
    return a

def print_superstring(strings):
    print('\n---BEGIN PRINTING SCS---')
    a = make_overlap_graph(strings)
    path = max_ATSP(a)
    lenght = len(strings[path[0]])
    print(strings[path[0]],end='')
    for i in range(1, len(path)):
        lenght += len(strings[path[i]][int(a[path[i-1]][path[i]]):])
        print(strings[path[i]][int(a[path[i-1]][path[i]]):],end='')
    print()
    shift = 0
    print(strings[path[0]])
    for i in range(1, len(path)):
        shift += len(strings[path[i-1]]) - a[path[i-1]][path[i]]
        print(' '*int(shift), end = '')
        print(strings[path[i]])
        
    print('Length =',lenght)
    print('Compression =',path_summ(a, path))
    
    print('----END PRINTING SCS----\n')
    return True

def make_optimal_cycle(strings):
    a = make_overlap_graph(strings)
    n = len(strings)
    if n <= 1:
        return []
    ans_val = 0
    ans_path = []
    path = []
    for per in permutations(range(n)):
        cur_val = 0
        for v in range(n):
            cur_val += a[per[v]][per[(v+1)%n]]
        if(ans_val <= cur_val):
            ans_val = cur_val
            ans_path = per
    for iistr in range(len(ans_path)):
        str1 = strings[ans_path[iistr]]
        str2 = strings[ans_path[(iistr + 1)%n]]
        over = str2[:overlap(str1,str2)]
        path = path + [str1[i:] for i in range(len(str1) - len(over))] + [str2[:i] for i in range(len(over), len(str2))]
    path.append(path[0])
    i_mi = 0
    for i in range(len(path)):
        if len(path[i])<len(path[i_mi]):
            i_mi = i
    if len(path[i_mi])!=0:
        addy = [path[i_mi][i:] for i in range(len(path[i_mi])+1)] + [path[i_mi][:i] for i in range(1,len(path[i_mi]))]
        path = path[:i_mi]+addy+path[i_mi:]
    for ip in range(len(path)):
        if path[ip] =='':
            path[ip] = Eps
    return [[path[i],path[i+1]] for i in range(len(path)-1)]

In [ ]:
lexicographical = True

def exp(input_strings):
    global picture
    global output
    output = '-'*(sum(len(s) for s in input_strings)+1)
    global answer
    answer = ''
    global Hint#
    global OPT_string #
    global exist
    exist.clear()
    global cycles
    cycles.clear()
    global filename
    global images
    images.clear()
    global filenumber
    filenumber = 0
    global V
    V = make_all_substrings_list(input_strings)[::-1]
    global mrk
    global E
    E.clear()
    global path
    path.clear()
    global OPT
    OPT.clear()
    global fall_edges
    fall_edges.clear()
    global Hint_stage
    Hint_stage = 0
    global color_only_fall
    color_only_fall = False
    global Eps, MaxLength
    MaxLength = max(len(st) for st in input_strings)
    HG = make_hierarchical_graph(input_strings)
    #HG.draw('file.ps',prog='circo')
    HG.layout('dot')
    mrk = [0]*len(V)
    
    Vsorted = []
    if lexicographical:
        Vsorted = sorted(list(V), key=lambda x: (-len(x) if x!=Eps else 0, x))
    else:
        Vsorted = sorted(list(V), key=lambda x: [float(y) for y in HG.get_node(x).attr['pos'].split(',')][::-1])[::-1]
    V = Vsorted
    
    #Make first edges for Greedy from input strings
    for iv in range(len(input_strings)):
        mrk[V.index(input_strings[iv])] = iv+1
        add_edge(HG.out_edges(input_strings[iv])[0])
        add_edge(HG.in_edges(input_strings[iv])[0])
    ###Begin Greedy###
    
    #Make Greedy edges
    for v in Vsorted:
        iv = V.index(v)
        length = leng(v)
        ball = ballance(E, v)
        if ball > 0:
            for e in HG.in_edges(v):
                if leng(e[0]) < leng(v):
                    ball-=1
                    add_edge(e)
                    if ball == 0:
                        break
            while ball > 0:
                for e in HG.in_edges(v):
                    if leng(e[0]) < leng(v):
                        ball-=1
                        add_edge(e, True)
                        if ball == 0:
                            break

        elif ball < 0:
            for e in HG.out_edges(v):
                if leng(e[1]) < leng(v):
                    ball+=1
                    add_edge(e)
                    if ball == 0:
                        break
            while ball < 0:
                for e in HG.out_edges(v):
                    if leng(e[1]) < leng(v):
                        ball+=1
                        add_edge(e, True)
                        if ball == 0:
                            break
        iu = iv
        
        
        #Check connectivity
        if len([iw for iw in range(iu+1, len(mrk)) if mrk[iu] == mrk[iw] and leng(V[iu]) >= leng(V[iw])]) == 0:
            if [mrk[iu] == mrk[iw] or 0 == mrk[iw] for iw in range(len(mrk))].count(False) != 0 and mrk[iu] != 0:
                if is_eulerian([V[iw] for iw in range(len(V)) if mrk[iw] == mrk[iu]], [list(e) for e in E]):
                    breaked = False
                    for e in HG.out_edges(V[iu]):
                        if not e in E and leng(e[1]) < leng(V[iu]):
                            add_edge(e)
                            breaked = True
                            break
                    if not breaked:
                        for e in HG.out_edges(V[iu]):
                            if leng(e[1]) < leng(V[iu]):
                                add_edge(e, True)
                                break
                    breaked = False
                    for e in HG.in_edges(V[iu]):
                        if not e in E and leng(e[0]) < leng(V[iu]):
                            add_edge(e)
                            breaked = True
                            break
                    if not breaked:
                        for e in HG.in_edges(V[iu]):
                            if leng(e[0]) < leng(V[iu]):
                                add_edge(e, True)
                                break

        #frame(V[iv], greedy_build_color)
    
    i_mi = 0
    notsu = True
    for i in range(len(E)):
        if E[i][1]=='_':
            notsu = False
            break
        if len(E[i][1]) < len(E[i_mi][1]):
            i_mi = i
    if notsu:
        all_min = [x for x in Vsorted if len(x)==len(E[i_mi][1])]
        mimi = E[i_mi][1]
        for st in reversed(all_min):
            iss = False
            iiss = -1
            for i in range(len(mimi)):
                if e[1] == st:
                    iss = True
                    iiss = i
                    break
            if iss:
                i_mi = iiss
                mimi = st
                break
        addy = []
        for i in range(len(mimi)):
            addy.append((mimi[i:],mimi[i+1:]))
        for i in range(len(mimi)):
            addy.append((mimi[:i],mimi[:i+1]))
        E = E[:i_mi]+addy+E[i_mi:]
    
    #End making Greedy edges
    #Hint_stage += 1
    ##Begin find Greedy path
    #V_set = set()
    #for e in E:
    #    V_set.add(e[0])
    #V = list(V_set)
    #path = [V[0]]
    #out_len = len(output)
    #out_i = 0
    #for v in V:
    #    if(leng(v) < leng(path[0])):
    #        path[0] = v
    #exist = [True]*len(E)
    #while exist.count(True) > 0:
    #    for ie in range(len(E)):
    #        e = E[ie]
    #        v = path[-1]
    #        if e[0] == v and exist[ie]:
    #            exist[ie] = False
    #            VV = [w for w in V if len([iw for iw in range(len(E)) if w in E[iw] and exist[iw]])]
    #            if exist.count(True) == 0 or len(VV) == len(weak_component(VV,[list(E[ie]) for ie in range(len(E)) if exist[ie]],VV[0])):
    #                path.append(e[1])
    #                if leng(e[0]) < leng(e[1]):
    #                    answer +=path[-1][-1]
    #                break
    #            else:
    #                exist[ie] = True
#
    #    #frame(path[-1], greedy_path_color)
    ##pause()
    ####End Greedy###
    
    Hint_stage += 1
    ###Make optimal string###
    OPT1 = make_optimal_cycle(input_strings+[''])
    for e in OPT1:
        OPT.append(e)
    pause()
    Hint_stage += 1
    OPT_len = len(OPT1)
    Greedy_len = len(E)
    
    ####Begin cycle cover
    #cycles = []
    #for v in input_strings:
    #    cycles.append(HG.out_edges(v)[0])
    #    cycles.append(HG.in_edges(v)[0])
    #    #frame(v, cycle_color)
#
#
    #V = Vsorted
    #for v in Vsorted:
    #    length = leng(v)
    #    ball = ballance(cycles, v)
    #    if ball > 0:
    #        for e in HG.in_edges(v):
    #            if leng(e[0]) < leng(v):
    #                ball-=1
    #                cycles.append(e)
    #                if ball == 0:
    #                    break
    #        while ball > 0:
    #            for e in HG.in_edges(v):
    #                if leng(e[0]) < leng(v):
    #                    ball-=1
    #                    cycles.append(e)
    #                    if ball == 0:
    #                        break
#
    #    elif ball < 0:
    #        for e in HG.out_edges(v):
    #            if leng(e[1]) < leng(v):
    #                ball+=1
    #                cycles.append(e)
    #                if ball == 0:
    #                    break
    #        while ball < 0:
    #            for e in HG.out_edges(v):
    #                if leng(e[1]) < leng(v):
    #                    ball+=1
    #                    cycles.append(e)
    #                    if ball == 0:
    #                        break
    #    #frame(v, cycle_color)
    
    #Hint_stage += 1
    #
    fall_edges_tmp = OPT + OPT
    fall_edges = [[e[0], e[1]] for e in fall_edges_tmp]
    #color_only_fall = True

    #Hint_stage += 1

    for v in Vsorted:
        #frame(v, falling_color)
        while len(edges_down_out(fall_edges, v))*len(edges_down_in(fall_edges, v)) > 0:
            e_l = edges_down_in(fall_edges, v)[0]
            e_r = edges_down_out(fall_edges, v)[0]
            v_l = e_l[0]
            v_r = e_r[1]
            v_d = v_l[1:]
            if v_l == v_r == Eps:
                cont = len([e for e in fall_edges if (e[0]==Eps or e[0]=='')])
                #print(v,cont)
                if cont == 1:
                    break
            if v_d == '':
                v_d = Eps
            if v in input_strings and len(edges_out(fall_edges, v))*len(edges_in(fall_edges, v)) == 1:
                #frame(v, falling_color)
                break
            elif len(edges_out(fall_edges, v))*len(edges_in(fall_edges, v)) == 1:
                if len(edges_down_out(fall_edges, v))*len(edges_down_in(fall_edges, v)) == 1:
                    if v_l == v_r:
                        if v_l == Eps or v_l == '':
                            cont = len([e for e in fall_edges if (e[0]==Eps or e[0]=='')])
                            if cont > 1:
                                fall_edges.remove(e_l)
                                fall_edges.remove(e_r)
                            else:
                                pass
                        else:
                            fall_edges.remove(e_l)
                            fall_edges.remove(e_r)
                    else:
                        fall_edges.remove(e_l)
                        fall_edges.append([v_l, v_d])
                        fall_edges.remove(e_r)
                        fall_edges.append([v_d, v_r])
                else:
                    #frame(v, falling_color)
                    break
            else:
                
                tmp = [[e[0], e[1]] for e in fall_edges]
                tmp.remove(e_l)
                tmp.remove(e_r)
                if v_l != v_d:
                    tmp.append([v_l, v_d])
                    tmp.append([v_d, v_r])
                V_set = set()
                for e in tmp:
                    V_set.add(e[0])
                if is_eulerian(list(V_set), tmp):
                    fall_edges = [copy.copy(e) for e in tmp]
                else:
                    #frame(v, falling_color)
                    break
            #frame(v, falling_color)
  
    
    pset1 = [list(e) for e in E]
    pset2 = [list(e) for e in fall_edges]
    mark2 = [False]*len(pset2)
    for i in range(len(pset1)):
        if len(pset1[i][1])==0:
            pset1[i][1]=Eps
        if pset1[i][0]=='':
            pset1[i][0]=Eps
        for j in range(len(pset2)):
            if pset1[i] == pset2[j] and not mark2[j]:
                mark2[j] = True
                break
    coll_conj = len(pset1)==len(pset2) and mark2.count(False) == 0
    
    if picture:
        frame2('none')
        frame2('OPT')
        frame2('GREEDY')
        frame2('FALL')
    return [Greedy_len, OPT_len, coll_conj]
picture = True
exp(['abab', 'baab', 'baba'])

In [ ]:
def gen_all(s):
    ans = []
    for i1 in range(s):
        for i2 in range(s):
            for i3 in range(s):
                for i4 in range(s):
                    ans.append(chr(ord('a')+i1)+chr(ord('a')+i2)+chr(ord('a')+i3)+chr(ord('a')+i4))
    return ans
ally = []

def gen11(n):
    while True:
        ans = []
        while len(ans) < n:
            st = ally[randint(0, len(ally)-1)]
            if ans.count(st)==0:
                ans.append(st)
        let = set()
        for st in ans:
            let.update(set(st))
        if len(let) != s:
            continue
        return sorted(ans)

def gen21(n, s, k):
    L = ''
    while len(L) < 4*n*k:
        L+=chr(ord('a')+randint(0, s-1))
    ans = []
    L1 = L+L[:5]
    while len(ans) < n:
        pos = randint(0, len(L)-1)
        st = L1[pos:pos+4]
        if ans.count(st)==0:
            ans.append(st)
    return sorted(ans)
    
def gen22(n, s, k):
    while True:
        cont = 0
        L = ''
        while len(L) < 4*n*k:
            L+=chr(ord('a')+randint(0, s-1))
        ans = []
        L1 = L+L[:5]
        pos = 0
        while len(ans) < n:
            st = L1[pos:pos+4]
            if ans.count(st)==0:
                ans.append(st)
            pos= (pos + randint(1, 3))%len(L)
            cont+=1
            if cont > n**3:
                break
        let = set()
        for st in ans:
            let.update(set(st))
        if cont > n**3 or len(let) != s:
            continue
        return sorted(ans)
breaky = False

In [ ]:
picture = False #for multystart
for s in range(2, 10):
    if breaky:
        break
    ally = gen_all(s)
    for n in range(3, 10):
        if breaky:
            break
        col1,col2,col3 = 0,0,0
        max1,max2,max3 = 0,0,0
        maxs1,maxs2,maxs3 = [],[],[]
        all_time = time.time()
        while (time.time() - all_time) < 500:
            lis = gen11(n)
            ans = exp(lis)
            prop = ans[0]/ans[1]
            if prop > 1: #now watching only on |GREEDY|/|OPT|
                col1+=1
                if prop > max1:
                    maxs1 = lis
        all_time = time.time()
        while (time.time() - all_time) < 500:
            lis = gen21(n,s,66/100)
            ans = exp(lis)
            prop = ans[0]/ans[1]
            if prop > 1: #now watching only on |GREEDY|/|OPT|
                col2+=1
                if prop > max2:
                    maxs2 = lis
        all_time = time.time()
        while (time.time() - all_time) < 500:
            lis = gen22(n,s,66/100)
            ans = exp(lis)
            prop = ans[0]/ans[1]
            if prop > 1: #now watching only on |GREEDY|/|OPT|
                col3+=1
                if prop > max3:
                    maxs3 = lis
        print(n, s)
        print("GEN1 :",max1,'\t',maxs1)
        print("GEN21:",max2,'\t',maxs2)
        print("GEN21:",max3,'\t',maxs3)
        print('---\n')